# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5380958ca0>
├── 'a' --> tensor([[ 1.0214, -0.1645,  0.6355],
│                   [ 0.2042,  0.2523,  0.5081]])
└── 'x' --> <FastTreeValue 0x7f53809587c0>
    └── 'c' --> tensor([[ 3.1689,  0.3907,  0.1241, -0.8749],
                        [ 1.7390,  0.3995,  1.7836,  0.6277],
                        [ 1.2627,  0.5921,  0.4079, -0.1474]])

In [4]:
t.a

tensor([[ 1.0214, -0.1645,  0.6355],
        [ 0.2042,  0.2523,  0.5081]])

In [5]:
%timeit t.a

65.3 ns ± 0.0805 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5380958ca0>
├── 'a' --> tensor([[-1.1399, -0.1353,  0.2886],
│                   [ 0.5824, -0.2276, -1.2670]])
└── 'x' --> <FastTreeValue 0x7f53809587c0>
    └── 'c' --> tensor([[ 3.1689,  0.3907,  0.1241, -0.8749],
                        [ 1.7390,  0.3995,  1.7836,  0.6277],
                        [ 1.2627,  0.5921,  0.4079, -0.1474]])

In [7]:
%timeit t.a = new_value

69.1 ns ± 0.1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.0214, -0.1645,  0.6355],
               [ 0.2042,  0.2523,  0.5081]]),
    x: Batch(
           c: tensor([[ 3.1689,  0.3907,  0.1241, -0.8749],
                      [ 1.7390,  0.3995,  1.7836,  0.6277],
                      [ 1.2627,  0.5921,  0.4079, -0.1474]]),
       ),
)

In [10]:
b.a

tensor([[ 1.0214, -0.1645,  0.6355],
        [ 0.2042,  0.2523,  0.5081]])

In [11]:
%timeit b.a

58.3 ns ± 0.298 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.0552, -2.7974, -0.5555],
               [ 0.3660, -1.4748, -0.5065]]),
    x: Batch(
           c: tensor([[ 3.1689,  0.3907,  0.1241, -0.8749],
                      [ 1.7390,  0.3995,  1.7836,  0.6277],
                      [ 1.2627,  0.5921,  0.4079, -0.1474]]),
       ),
)

In [13]:
%timeit b.a = new_value

496 ns ± 0.129 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

842 ns ± 1.03 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 13.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

141 µs ± 731 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 668 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f52c9c10610>
├── 'a' --> tensor([[[ 1.0214, -0.1645,  0.6355],
│                    [ 0.2042,  0.2523,  0.5081]],
│           
│                   [[ 1.0214, -0.1645,  0.6355],
│                    [ 0.2042,  0.2523,  0.5081]],
│           
│                   [[ 1.0214, -0.1645,  0.6355],
│                    [ 0.2042,  0.2523,  0.5081]],
│           
│                   [[ 1.0214, -0.1645,  0.6355],
│                    [ 0.2042,  0.2523,  0.5081]],
│           
│                   [[ 1.0214, -0.1645,  0.6355],
│                    [ 0.2042,  0.2523,  0.5081]],
│           
│                   [[ 1.0214, -0.1645,  0.6355],
│                    [ 0.2042,  0.2523,  0.5081]],
│           
│                   [[ 1.0214, -0.1645,  0.6355],
│                    [ 0.2042,  0.2523,  0.5081]],
│           
│                   [[ 1.0214, -0.1645,  0.6355],
│                    [ 0.2042,  0.2523,  0.5081]]])
└── 'x' --> <FastTreeValue 0x7f52c9c10580>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.2 µs ± 56.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f52c9cf4850>
├── 'a' --> tensor([[ 1.0214, -0.1645,  0.6355],
│                   [ 0.2042,  0.2523,  0.5081],
│                   [ 1.0214, -0.1645,  0.6355],
│                   [ 0.2042,  0.2523,  0.5081],
│                   [ 1.0214, -0.1645,  0.6355],
│                   [ 0.2042,  0.2523,  0.5081],
│                   [ 1.0214, -0.1645,  0.6355],
│                   [ 0.2042,  0.2523,  0.5081],
│                   [ 1.0214, -0.1645,  0.6355],
│                   [ 0.2042,  0.2523,  0.5081],
│                   [ 1.0214, -0.1645,  0.6355],
│                   [ 0.2042,  0.2523,  0.5081],
│                   [ 1.0214, -0.1645,  0.6355],
│                   [ 0.2042,  0.2523,  0.5081],
│                   [ 1.0214, -0.1645,  0.6355],
│                   [ 0.2042,  0.2523,  0.5081]])
└── 'x' --> <FastTreeValue 0x7f52c9cf4880>
    └── 'c' --> tensor([[ 3.1689,  0.3907,  0.1241, -0.8749],
                        [ 1.7390,  0.3995,  1.7836,  0.6277],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 136 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.1 µs ± 126 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.0214, -0.1645,  0.6355],
                [ 0.2042,  0.2523,  0.5081]],
       
               [[ 1.0214, -0.1645,  0.6355],
                [ 0.2042,  0.2523,  0.5081]],
       
               [[ 1.0214, -0.1645,  0.6355],
                [ 0.2042,  0.2523,  0.5081]],
       
               [[ 1.0214, -0.1645,  0.6355],
                [ 0.2042,  0.2523,  0.5081]],
       
               [[ 1.0214, -0.1645,  0.6355],
                [ 0.2042,  0.2523,  0.5081]],
       
               [[ 1.0214, -0.1645,  0.6355],
                [ 0.2042,  0.2523,  0.5081]],
       
               [[ 1.0214, -0.1645,  0.6355],
                [ 0.2042,  0.2523,  0.5081]],
       
               [[ 1.0214, -0.1645,  0.6355],
                [ 0.2042,  0.2523,  0.5081]]]),
    x: Batch(
           c: tensor([[[ 3.1689,  0.3907,  0.1241, -0.8749],
                       [ 1.7390,  0.3995,  1.7836,  0.6277],
                       [ 1.2627,  0.5921,  0.4079, -0.1474]],
         

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 88.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.0214, -0.1645,  0.6355],
               [ 0.2042,  0.2523,  0.5081],
               [ 1.0214, -0.1645,  0.6355],
               [ 0.2042,  0.2523,  0.5081],
               [ 1.0214, -0.1645,  0.6355],
               [ 0.2042,  0.2523,  0.5081],
               [ 1.0214, -0.1645,  0.6355],
               [ 0.2042,  0.2523,  0.5081],
               [ 1.0214, -0.1645,  0.6355],
               [ 0.2042,  0.2523,  0.5081],
               [ 1.0214, -0.1645,  0.6355],
               [ 0.2042,  0.2523,  0.5081],
               [ 1.0214, -0.1645,  0.6355],
               [ 0.2042,  0.2523,  0.5081],
               [ 1.0214, -0.1645,  0.6355],
               [ 0.2042,  0.2523,  0.5081]]),
    x: Batch(
           c: tensor([[ 3.1689,  0.3907,  0.1241, -0.8749],
                      [ 1.7390,  0.3995,  1.7836,  0.6277],
                      [ 1.2627,  0.5921,  0.4079, -0.1474],
                      [ 3.1689,  0.3907,  0.1241, -0.8749],
                      [ 1.7390,  

In [28]:
%timeit Batch.cat(batches)

144 µs ± 1.09 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

328 µs ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
